In [1]:
# -*- coding: utf-8 -*-
import sys
import codecs
import json

In [2]:
import requests
from bs4 import BeautifulSoup
from http.server import BaseHTTPRequestHandler, HTTPServer
import urllib.parse as urlparse

In [3]:
link = "https://batdongsan.vn/"
t = "/ban-nha/p"
r = requests.get(link+t[0])
soup = BeautifulSoup(r.text, 'html.parser')




In [7]:
def find_link(soup, link):
    l = []
    div_items = soup.find_all("div", attrs={"class" : "item"})
    for div_item in div_items:
        a_links = div_item.find_all('a',href=True)
        for a_link in a_links:
            l.append(a_link['href'])
    return l

# for value in find_link(soup, link):
#     print(value)

https://batdongsan.vn/ban-dat-o-binh-phan-1000m2-cho-gao-tien-giang-gan-bo-song-r285160
https://batdongsan.vn/ban-dat-o-binh-phan-1000m2-cho-gao-tien-giang-gan-bo-song-r285160
https://batdongsan.vn/ban-dat
https://batdongsan.vn/nguyenthanhdanh.199095
#
https://batdongsan.vn/chu-gap-ban-truoc-tet-le-hong-phong-quan-5-ra-mat-tien-duong-15m-r285161
https://batdongsan.vn/chu-gap-ban-truoc-tet-le-hong-phong-quan-5-ra-mat-tien-duong-15m-r285161
https://batdongsan.vn/ban-nha-rieng
https://batdongsan.vn/briandonghan
#
https://batdongsan.vn/sat-mat-tien-phan-dang-luu-phuong-7-phu-nhuan-2-tang-btct-45m2-4x-ty-r285162
https://batdongsan.vn/sat-mat-tien-phan-dang-luu-phuong-7-phu-nhuan-2-tang-btct-45m2-4x-ty-r285162
https://batdongsan.vn/ban-nha
https://batdongsan.vn/chauduy
#
https://batdongsan.vn/ban-nha-hxh-au-co-phuong-9-tan-binh-51m2-3-tang-gia-5-ty-nhinh-r285163
https://batdongsan.vn/ban-nha-hxh-au-co-phuong-9-tan-binh-51m2-3-tang-gia-5-ty-nhinh-r285163
https://batdongsan.vn/ban-nha-rieng
ht

In [5]:
def find_data(soup):
    data = {}

    # Tiêu đề
    titles = soup.find_all('div', {"class": "name"})
    data['title'] = [title.find('a').text.strip() for title in titles]

    # Giá
    prices = soup.find_all('span', {"class": "price"})
    data['price'] = [price.text.strip() for price in prices]

    # Diện tích
    acreages = soup.find_all('span', {"class": "acreage"})
    data['acreage'] = [acreage.text.strip() for acreage in acreages]

    # Mô tả
    descriptions = soup.find_all('div', {"class": "sapo"})
    data['description'] = [description.find('p').text.strip() for description in descriptions]

    return data

def process_property_elements(soup):
    data_list = []

    # Tìm tất cả các phần tử HTML chứa thông tin
    property_elements = soup.find_all('div', {"class": "item"})

    for property_element in property_elements:
        # Gọi hàm find_data cho mỗi phần tử và thêm kết quả vào danh sách
        data_list.append(find_data(property_element))

    return data_list

def scrape_and_save(link, base_link, t, max_pages):
    all_data = []

    for i in range(1, max_pages + 1):
        try:
            r = requests.get(link + t + str(i))
            r.raise_for_status()  # Raise HTTPError for bad requests
            soup = BeautifulSoup(r.text, 'html.parser')
            
            property_data = process_property_elements(soup)
            all_data.extend(property_data)

        except requests.RequestException as e:
            print(f"Error fetching page {i}: {e}")

    with open('bat_dong_san.json', 'w') as file:
        json.dump(all_data, file)

    print("Done")

In [6]:
max_pages = 100

scrape_and_save(link, link, t, max_pages)

Done
